**ANSWER:** when it's a holiday or weekend, the trend is reversed for bike usage among reg vs casual. There's higher usage on weekdays with reg (probably use it for biking to work) vs weekends for casuals who probably want to have fun on weekends.
Usage Trend for reg vs casual is the same in the year (peak on June-July) and highest when weather is best (1 vs 3).

## 2. The Regression Model <a id= "section 2"></a>

To try to predict the number of riders on a given day, we'll use a regression model. From Module II:

> A **simple regression model** describes how the conditional mean of a response variable $y$ depends on an explanatory variable $x$: $$\mu_{y|x} = \beta_0 + \beta_1x$$ This equation describes our best guess for $y$ given a particular  $x$.

> A **multiple regression model** describes how the conditional mean of a response variable $y$ depends on multiple explanatory variables $x_1, x_2, ..., x_k$: $$\mu_{y|x_1, x_2, ..., x_k} = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k$$ This equation describes our best guess for $y$ given a particular vector $x_1, x_2, ..., x_k$.

In this case, our model will look something like this:
$$\mu_{\text{rider count}|\text{temp, humidity, ..., month}} = \beta_0 + \beta_1*\text{(temp)} + \beta_2*\text{(humidity)} + ... + \beta_k*\text{(month)}$$
The code for either a simple or multiple regression model is basically identical except for the number of columns we select for inclusion in our explanatory variables.

To create our model, we need to:
1. isolate the explanatory variables (X) and response variable (y) we want to use
2. find the values for the $\beta$ variables on the best-fit regression line
3. evaluate the accuracy of our model

### 2a. Explanatory and response variables <a id="subsection 2a"></a>

First, let's decide on our response variable. We'll try to predict the *total number of riders* on a given day. The response variable needs to be in an array (not a Table), so we'll get it using the `column` function.

In [ ]:
# response variable: total count of riders in a day (training set)
y_train = bike_train['total riders']

# response variable: total count of riders in a day (validation set)
y_test = bike_test['total riders']

y_train

Next, we want to choose our explanatory variables. Let's try predicting ridership in terms of _temperature_, _work day_, and _season_.

<div class="alert alert-info">

**Why don't we just use all the explanatory variables?**
You might think that the best model would use *all* the available explanatory information. But, using many variables makes a model **computationally expensive**. In the real world, where data sets may have a million or more rows, using a complex model can increase the time and computing power needed to make preditions. Additionally, many variables may have **little predictive power** such that excluding them from the model doesn't lower the accuracy very much. Other variables might **add noise** that actually decreases the model's performance outside of the training data.

</div> 

Here, we run into a problem: "work day" and "season" are categorical variables (even though they have numerical values). This gets tricky with regression- the computer starts to work with the values as if they were numbers, and that can lead to questionable manipulations. For example, since Sunday is coded as 0 and Saturday is coded as 6, the computer might conclude that the average of Sunday and Saturday is Wednesday (since Wednesday is coded as 3).

#### One-Hot Encoding
To work around this, we will **one-hot encode** all our categorical variables. In one-hot encoding, the possible values of the variable each get their own column of 1s and 0s. The value is a 1 if that day falls in that category and a 0 otherwise.

Here's an example. Say we have three possible weather states: rain, cloudy, or sunny.

In [ ]:
# original statement: categorical = Table().with_columns("weather", ["rainy", "cloudy", "sunny", "cloudy"])
# corrected statement below eliminates duplicated value of "cloudy"
categorical = Table().with_columns("weather", ["rainy", "cloudy", "sunny"])
categorical

The one-hot encoding would look like this:

In [ ]:
# pd.get_dummies is a function that does dummy encoding
one_hot = pd.get_dummies(categorical.column('weather'))
one_hot

Notice that in each row, only one of the values is equal to 1 (hence the name, "one-hot" encoding), since no day can have more than one weather state.

Notice also that we don't technically need the third column. 

In [ ]:
# This was original statement which I changed below: one_hot.drop("sunny", axis=1)
one_hot = one_hot.drop("sunny", axis=1)
one_hot

If we know that there are only three possible weather states, and we see that day 2 was neither cloudy nor rainy (that is, `cloudy`=0 and `rainy`=0), day 2 *must* have been sunny. This is helpful to save computation time and space. If you have some knowledge of linear algebra, note that this is also helpful to solve the problem of *perfect multicollinearity*- a situation that can make it impossible to compute the optimal set of $\beta$s.

For simplicity, we've provided a function called `format_X` that will take a Table of explanatory variables and convert it to the correct format for prediction, including one-hot encoding the categorical variables. `format_X` will also add a column called "intercept" that only contains 1s. This column will help find the intercept term $\beta_0$ in our regression line. You can think of the intercept term as an $x_0$ that gets multiplied by $\beta_0$ and is always equal to 1.

In [ ]:
ds1 = bike_train['temp']
ds2 = bike_train['is work day']
ds3 = bike_train['season']
ds3 = pd.get_dummies(ds3)

X_train = pd.concat([ds1, ds2, ds3], axis=1)
X_train = X_train.drop(labels='4:winter', axis=1)
X_train.insert(0, 'intercept', 1)
X_train.head()

**EXERCISE:** Since we want to try the model on the test data as well, we will also perform the same transformations on the test set so it can fit the model. Fill in the code to first select the explanatory variables "temp", "is work day", and "season", then convert the explanatory table to the matrix format using `format_X`. Hint: we'll need to go through the exact same steps as in the above cell for the training data, but any references to training data should be replaced by their test data counterparts.

In [ ]:
ds1 = bike_test['temp']
ds2 = bike_test['is work day']
ds3 = bike_test['season']
ds3 = pd.get_dummies(ds3)

X_test = pd.concat([ds1, ds2, ds3], axis=1)
X_test = X_test.drop(labels='4:winter', axis=1)
X_test.insert(0, 'intercept', 1)
X_test.head()

### 2b.  Finding $\beta$ <a id="subsection 2b"></a>
The next step is to calculate the $\beta$ terms. We can do this with a function from the Scipy statistical analysis Python package called `lstsq`.

Given a matrix of explanatory variables X and an array of response variables y, `lstsq` returns a vector $\beta$. `lstsq` uses *ordinary least squares* as its **loss function (L)**: the function that defines the training loss (error) and what we seek to minimize (often using linear algebra or calculus, depending on the loss function). The ordinary least squares equation is a common loss function that is used to minimize the sum of squared errors:

$$L(\beta) = \frac{1}{n}\sum_{n}^{i=1}(y_i - \text{predicted}(\beta, x_i))^2$$

where $n$ is the number of days, $y_i$ is the actual number of riders on the $i$th day, and $\text{predicted}(\beta, x_i)$ is number of riders predicted to be on the $i$th day when using $\beta$ and the explanatory variables $x_i$. When minimized, the loss function will yield our optimal $\beta$. 

`lstsq` returns a list of four things, but for our purposes we're only interested in one of them: the array of the $\beta$ values for the best-fit line.

In [ ]:
# calculate the least squares solution
# Apply the Training dataset with 6 explanatory variables (X_train Array containing: intercept, temp, is work day, 1:spring, 2:summer, 3:fall)
# Input 2nd parameter to lstsq function with Response variable y_train which is "total riders"
y_train = bike_train['total riders']
lstsq_results = lstsq(X_train, y_train)
print(lstsq_results)

# extract row 0 from function scipy.linalg.lstsq which returns 4 rows
beta = lstsq_results[0]
beta

We now have everything we need to make predictions about the total number of riders on a given day. Remember, the formula is: $$\mu_{y|x_1, x_2, ..., x_k} = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k$$

So, to make a prediction for a given day, we take all the values in the X matrix corresponding to that day, multiply each value by its corresponding $\beta$, then add them all together. The `@` operator can help us with this matrix multiplication.

For example, here's the first row of our explanatory variable matrix.

**EXERCISE:** We also want to make predictions for the test data using the $\beta$ we found during training. Replace the `...` in the cell below with an expression to calculate the predictions for the test set. Remember- you need to use `@` to multiply each row of explanatory variables in our test set by the $\beta$ vector. Look at how `predict_train` was calculated for a guide.

In [ ]:
y_test = bike_test['total riders']
lstsq_results_test = lstsq(X_test, y_test)
print(lstsq_results_test)
beta = lstsq_results_test[0]
beta

In [ ]:
X_test.loc[0:9, :]
X_test.loc[0, :] @ beta

predict_test = X_test @ beta
predict_test

bike_test.insert(16, "predicted total riders", predict_test)
bike_test

### 2c. Evaluating the model <a id="subsection 2c"></a>

Our model makes predictions, but how good are they? We can start to get a sense of how we did by plotting the predictions versus the actual values on our training data on a scatter plot. Remember from Module II that if our model predicts perfectly:

- the predicted values will be equal to the actual values
- all points in the scatter plot will fall along a straight line with a slope of 1

As a bonus, `scatter` has an optional argument called `fit_line` that will add the best-fit line to the plot if you mark it as `True`.

In [ ]:
df = bike_train
sns.lmplot(x='predicted total riders',y='total riders',data=df,fit_reg=True) 
plt.title("Training set predictions")


Here are the validation set predictions scattered against the actual total rider counts. Note that we've added an extra `color` argument to `scatter` to change the color of the dots and line and distinguish the validation data from the training data.

In [ ]:
df = bike_test
sns.lmplot(x='predicted total riders',y='total riders',data=df,fit_reg=True) 
plt.title("Test set predictions")


We can also get a quantitative measure of how good our model is by calculating the **root mean squared error**. This is fairly straightforward to calculate now that we have our error vector:
1. subtract the predictions from the actual values to get the errors
2. square the errors
3. take the average of the squared errors
4. take the square root of the average squared error

In [ ]:
# the root mean squared error of the training data
errors = y_train - predict_train
sq_error = errors ** 2
mean_sq_error = np.average(sq_error)
root_mean_sq_err = np.sqrt(mean_sq_error)
root_mean_sq_err

<div class="alert alert-info">

**Why Root Mean Squared Error (RMSE)?**
To know why RMSE is useful, it helps to think about the steps to derive it in order. We *square* the errors to get rid of any negative signs (we don't care if we overpredict or underpredict, we just want to know the magnitude of the error). We then want the *average* magnitude of the error to see how far off our model typically was. Finally, we take the *square root* to get back to the original units (in this case, number of riders as opposed to squared number of riders).

</div> 

Next, we want to see what the RMSE is for our test set predictions. To simplify things, we have a function called `rmse` that takes a column of predicted values and a column of actual values and calculates the root mean squared error.

Before you run the next cell: would you expect the RMSE for the test set would be higher, lower, or about the same as the RMSE for the training set? Why?

In [ ]:
errors = y_test - predict_test
sq_error = errors ** 2
mean_sq_error = np.average(sq_error)
root_mean_sq_err = np.sqrt(mean_sq_error)
root_mean_sq_err

We can also visualize our errors compared to the actual values on a scatter plot.

[View in Colaboratory](https://colab.research.google.com/github/ruzbro/data-analysis/blob/master/03LinearRegressionModels_converted_from_JupyterHyub.ipynb)

# Berkeley Program on Data Science and Analytics
## Module IV, Part III: A Linear Regression Model

<br/>

<div class="container">
    <div style="float:left;width:40%">
	    <img src="images/bikeshare_sun.jpg">
    </div>
    <div style="float:left;width:40%">
	    <img src="images/bikeshare_snow.PNG">
    </div>
</div>

### Table of Contents

[Case Study: Bike Sharing](#section case)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   [The Test-Train Split](#subsection 0)


1 - [Exploratory Data Analysis](#section 1)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a - [Data Types and Summary Statistics](#subsection 1a)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b - [Visualizations Continued: Numerical Data and Widgets](#subsection 1b)


2 - [Linear Regression Model](#section 2)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a - [Explanatory and Response Variables](#subsection 2a)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b - [Finding $\beta$](#subsection 2b)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; c - [Evaluating the Model](#subsection 2c)


3 - [Challenge: Improve the Model](#section 3)<br>


In [ ]:
!pip install ipywidgets
!pip install --upgrade datascience

In [ ]:
# run this cell to import some necessary packages
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') # what is 538 ???
%matplotlib inline

#from scripts.exec_ed_scripts import *
import pandas as pd
import ipywidgets as widgets
from scipy.linalg import lstsq
from scipy.linalg import lstsq
import seaborn as sns

In [ ]:
!pip freeze

## Case Study: Capital Bike Share <a id= "section case"></a>

Bike-sharing systems have become increasingly popular worldwide as environmentally-friendly solutions to traffic congestion, inadequate public transit, and the "last-mile" problem. Capital Bikeshare runs one such system in the Washington, D.C. metropolitan area.

The Capital Bikeshare system comprises docks of bikes, strategically placed across the area, that can be unlocked by *registered* users who have signed up for a monthly or yearly plan or by *casual* users who pay by the hour or day. They collect data on the number of casual and registered users per hour and per day.

Let's say that Capital Bikeshare is interested in a **prediction** problem: predicting how many riders they can expect to have on a given day. [UC Irvine's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) has combined the bike sharing data with information about weather conditions and holidays to try to answer this question.

In this notebook, we'll walk through the steps a data scientist would take to answer this question.

###  The Test-Train Split <a id="subsection 0"> </a>

When we train a model on a data set, we run the risk of [**over-fitting**](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html). Over-fitting happens when a model becomes so complex that it makes very accurate predictions for the data it was trained on, but it can't generalize to make good predictions on new data.

Over- and under-fitting are most easily explained visually. The [Scikit-Learn machine learning library](http://scikit-learn.org) has a good example:

<img src="http://scikit-learn.org/stable/_images/sphx_glr_plot_underfitting_overfitting_001.png"/>

The linear model on the left is **under-fitting**: we can see that there is a lot of vertical distance (the *error*) between the actual samples (the dots) and the prediction (the blue line). The 15-degree model on the right is over-fitting: there's almost no error, but the model is so complex it is unlikely to generalize to new data. Our goal is to get the model in the middle: reduce the error as much as possible while keeping the complexity low.

We can reduce the risk of overfitting by using a **test-train split**. 

1. Randomly divide our data set into two smaller sets: one for training and one for testing
2. Train the data on the training set, changing our model along the way to increase accuracy
3. Test the data's predictions using the test set.

<div class="alert alert-info">

**Over-fitting to the test set**
By using the test set over and over to check the predictive accuracy of different models, we run the risk of over-fitting to the test set as well. In the real world, data scientists get around this by also using a **validation set**- a portion of training data that the model isn't trained on, used to find optimal *hyperparameters* for the model (parameters that are set before the model is trained). Because we're using only limited hyperparameters, and because our model is for educational purposes, we'll only use training and test sets.

</div>

Our bike data has been divided ahead of time into test and training sets. Run the next cell to load the training and test data.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# run this cell to load the data
bike_train = pd.read_csv("day_train.csv")

# load the test data
bike_test = pd.read_csv("day_test.csv")

# reformat the date column from strings to dates
bike_train['date'] = pd.to_datetime(bike_train['date'])
bike_test['date'] = pd.to_datetime(bike_test['date'])
bike_train.head()

**QUESTION:** Data is often expensive to collect, and having a good predictive model can be the difference between success and ruin. Given these factors, the decision of how much data to set aside for testing and validation is very personal.

What are some reasons for putting a larger portion of data into the training set? What are some risks?

**ANSWER:** The assumption would be that "more data" makes for comprehensive training set --> better predictions. Risks???

## 1. Exploratory Data Analysis (EDA) <a id= "section 1"></a>

> "It is important to understand what you CAN DO before you learn to measure how WELL you seem to have done it." -John W. Tukey, *Exploratory Data Analysis*

**Exploratory Data Analysis (EDA)** is the process of 'looking at data to see what it seems to say'. EDA is an essential first step toward answering any research question. Through this process, we hope to accomplish several things:
- learn about the overall 'shape' of the data: structure, organization, ranges of values
- assess what assumptions we can make about the data as a basis for later statistical inference
- figure out the appropriate tools and techniques for analysis 
- tentatively create testable, appropriate hypotheses or models

We will do this by looking at summary statistics and visualizations of the different variables.

### 1a. Data Types and Summary Statistics <a id= "subsection 1a"></a>

Before we even know how to visualize the data, we need to know what types of data we're working with. Run the following cell to show our bike sharing training data.

In [ ]:
bike_train

A few of the less straight-forward columns can be described as follows:
- **instant**: record index
- **is 2012** : 1 if the date is in 2012, 0 if the date is in 2011
- **is holiday** : 1 if day is a holiday, 0 otherwise
- **is work day** : 1 if day is not a weekend or holiday, otherwise 0
- **weather** :
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- **temp** : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
Note that historic temperature in Washington DC has exceeded 39C several times over the past decade
https://www.currentresults.com/Yearly-Weather/USA/DC/Washington/extreme-annual-washington-high-temperature.php

- **felt temp**: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- **humidity**: Normalized humidity. The values are divided to 100 (max)
- **windspeed**: Normalized wind speed. The values are divided to 67 (max)
- **casual**: count of casual users
- **registered**: count of registered users
- **total riders**: count of total rental bikes (casual + registered)

**QUESTION:** Which of the variables are numerical and which are categorical? Intuitively, which do you think would be useful for predicting the number of riders on a given day? Would you choose different variables depending on if you wanted to predict casual versus registered rider counts?

**ANSWER:**  
Which of the variables are numerical and which are categorical?
categorical: is 2012, is holiday / workday, weather... all the rest are numerical (excluding record index)

Intuitively, which do you think would be useful for predicting the number of riders on a given day?
Weather should be biggest predictor of # of riders (weather = 1 or 2)

Would you choose different variables depending on if you wanted to predict casual versus registered rider counts?
Registered riders may be more hardcore bike riders hence may be willing to bike even in inclement weather, unlike casual riders that may be more inclined when weather's great.

#### Summary Statistics
It can also be useful to know some *summary statistics* about the different variables: things like the minimum, maximum, and average. Earlier, we learned how to do this on individual columns using functions like `min`, `max`, and `np.average`.

Thankfully, we can generate a variety of summary statistics for many variables at the same time using a function called `describe`. `describe` works on a variety of table called a dataframe. Run the following cell to convert our bike data table to a dataframe, then generate the summary statistics.

In [ ]:
# generate summary statistics
bike_train.describe()

**QUESTION:** Looking at these statistics as data scientists, we're interested in a few things in particular:
- are there any values missing (e.g. days for which some data was not collected)?
- what kinds of values does each variable take?
- are there any extreme values that might throw off our analysis?

Using the summary table, answer these questions below.

**ANSWER:**
are there any values missing (e.g. days for which some data was not collected)? All columns contain 584 values.
what kinds of values does each variable take? numeric
are there any extreme values that might throw off our analysis? None.


### 1b. Visualization Continued: Numerical Data and Widgets <a id= "subsection 1b"></a>
So far, we've worked largely with categorical variables, which we visualized with bar graphs. The bike sharing data contains several *numerical* variables, which will necessitate different visualizations.

You've previously used the `hist` function to visualize the distribution of a numerical variable. The following cell creates a **widget** that will make different histograms based on the variable you choose in the drop box. Run the cell to created the widget (don't worry too much about the details of the code).

In [ ]:
#@title Histogram widget
# create a widget to plot and compare different histograms
# max values: temp: 39c, felt temp 50c, humidity: 100%, windspeed 67mph
explantory_slider = 'temp' #@param ["temp", "felt temp", "humidity", "windspeed"]
display(widgets.interactive(lambda x: bike_train.hist(x, bins=30), x=explantory_slider))

**QUESTION:** Describe the distributions of the different variables. Are they normally distributed? Are any of them *skewed* (that is, do any of them have most of their values to the left or right of the histogram's center)? What values do each of them take on?

**ANSWER:** temp and felt-temp seems to be multi-modal. with humidity and windspeed both normally distributed (except for a left skew on windspeed)

To predict the number of riders (the **response variable**) based on an **explanatory variable**, we often want to plot them against one another

`scatter` is a table function that creates a scatter plot of one numerical variable versus another. The first argument specifies the name of the variable to be plotted on the x-axis, and the second specifies the name of the variable on the y-axis.

In [ ]:
# example of scatter: plot the number of casual riders against registered riders
bike_train.plot.scatter("casual", "registered")

As you might remember from Professor Kariv's videos in Module II, the shape of the scatter plot can give us some information about how two variables are **correlated**: what the value of one variable can or cannot tell you about the value of another.

**EXERCISE:** Try plotting at least one numerical explanatory variable (temp, felt temp, windspeed, or humidity) against a response variable (casual, registered, or total riders). What would you say about the relationship between the two variables based on the scatter plot?

In [ ]:
# your code here
bike_train.plot.scatter("windspeed", "registered")

You can also *overlay* two scatter plots on top of one another. This can be helpful when comparing multiple responser variables.

To overlay two scatter plots, give a list of columns as the second argument instead of a single column name. The following cell gives an example: try substituting in different column names to create different plots.

In [ ]:
# plot date of the year against casual and total riders
df1 = bike_train.plot.scatter("temp", "casual", label="casual", color="y")
df2 = bike_train.plot.scatter("temp", "registered", label="registered", color = "b", ax=df1)

In [ ]:
# plot date of the year against casual and total riders
df1 = bike_train.plot.scatter("humidity", "casual", label="casual", color="y")
df2 = bike_train.plot.scatter("humidity", "registered",label="registered", color="b", ax=df1)

In [ ]:
# plot date of the year against casual and total riders
df1 = bike_train.plot.scatter("humidity", "casual", label="casual", color="y")
df2 = bike_train.plot.scatter("humidity", "total riders", label="total riders",color = "b", ax=df1)

In [ ]:
# plot date of the year against casual and total riders
df1 = bike_train.plot.scatter("windspeed", "casual", label="casual", color="y")
df2 = bike_train.plot.scatter("windspeed", "total riders", label="total riders", color = "b", ax=df1)

In the following cell, we've created another widget to make it easier to compare multiple variables against one another.

In [ ]:
# create a widget to make different scatter plots
x = 'humidity' #@param ["date", "temp", "felt temp", "humidity", "windspeed"]
df1 = bike_train.plot.scatter(x, "registered", label="registered", color="y")
df2 = bike_train.plot.scatter(x, "casual", label="casual", color="b", ax=df1)

**QUESTION:** Based on the scatter plots, which variables appear to be linearly correlated with rider counts? Which variables appear to be non-linearly correlated or uncorrelated? Is the apparent correlation different for casual or registered riders?

**ANSWER:** Temp and felt-temp seems to be linearly correlated. although there's less correlation with registered users -- which probably indicates that the registered are heavy users and unfazed by increase in temp.
Windspeed and Humidity doesn't seem to be correlated.

Finally, we want to visualize our categorical variables using bar graphs. Remember, for categorical variables we are grouping rows into the different possible categories (like the seven days of the week) and aggregating all the values in the group into a single value (in this case, taking the average). 

Run the next cell to create a widget for making the different bar plots.

In [ ]:
# define a function that groups by an explanatory variable and creates a horizontal bar graph
def barh_bikes(expl_var):
    g = bike_train[[expl_var, "registered", "casual"]] 
    g = g.groupby([expl_var]).agg(['mean'])
    return g.plot.barh()

explantory_slider = 'day of week' #@param ["season", "month", "is holiday", "day of week", "is work day", "weather"]
display(widgets.interactive(barh_bikes, expl_var=explantory_slider))

**QUESTION:** Many of our categorical variables are related to time (e.g. week day, month, etc). How do usage patterns over time differ for registered and casual users? In what categories do the different user types act similarly?

In [ ]:
X_train.loc[0:9, :]

In [ ]:
print(X_train)

To get the prediction, we use `@` to multiply each item in the row by each corresponding item in the $\beta$ vector and sum them all up. If you've taken linear algebra, you'll recognize this as the [*dot product*](https://en.wikipedia.org/wiki/Dot_product).

<img src="images/vector_mult.png" />



In [ ]:
# multiply the arrays using Row 0 of X_train to get the prediction
X_train.loc[0, :] @ beta

The `@` operator can also work on matrices. To get the predictions for *every* row in X, we use exactly the same syntax.

<img src="images/matrix_mult.png"  />


In [ ]:
predict_train = X_train @ beta
predict_train


Now we can add our predictions to our original table.

In [ ]:
bike_train.insert(16, 'predicted total riders', predict_train)
bike_train

In [ ]:
bike_train.insert(17, "training error", errors)
df = bike_train
sns.lmplot(x='predicted total riders',y='training error',data=df,fit_reg=True) 
plt.title("Training error set predictions")

In [ ]:
bike_test.insert(17, "validation error", y_test - predict_test)
df = bike_test
sns.lmplot(x='predicted total riders',y='validation error',data=df,fit_reg=True) 
plt.title("Test error set predictions")

**QUESTION:** Based on the plots and root mean squared error above, how well do you think our model is doing? What does the shape of the scatter plot of errors tell us about the appropriateness of the linear model here?

**ANSWER:** Slope of train vs predicted riders is < 0.885 (rmse of train / test = 1392 / 1572) and non-linear scatter plots shows that the model needs some improvement.

## 3. Challenge: Improve the Model <a id="section 3"></a>

Our model is currently not very good. But, there's a lot of things we could try to improve it.

In the following cells you have almost everything you need to create a new linear regression model. To try a new model, fill in the two sets of ellipses below:
- set `response` to the *string name* of the response variable you want to predict
- set `expl` to be a *list of string names of explanatory variables* you want to incorporate into the model. Remember, the names should be strings (i.e. in quotation marks) and separated by commas in between the square brackets.

Once you've filled in the ellipses, run all the cells below in order to recalculate the $\beta$ vector, make new predictions, and look at the residuals. A helpful tip: in the "Cell" menu at the top, clicking "Run All Below" will run all code cells below the cell you currently have selected.

How accurate can you make the model?